#### __pycache__ و __pynb_checkpoints__: اینا پوشه‌های سیستمی هستن که برای کش (cache) و نقاط چک ‌پوینت (checkpoints) جوپیتر نوت‌بوک استفاده می‌شن.


#### embedding:  مسئول تبدیل ورودی‌ها به بردارهای عددی (embeddings) برای مدل هست.


#### decoder و decoder_layer: بخش‌هایی از مدل ترنسفورمر که برای پردازش خروجی‌ها و پیش‌بینی بعدی‌ها استفاده می‌شن.

#### encoder و encoder_layer: بخش‌هایی از مدل که برای رمزگذاری ورودی‌ها و استخراج ویژگی‌ها به کار می‌رن.

#### encoder_feed_forward: لایه‌ای برای پردازش تغذیه رو به جلو در بخش رمزگذار.

#### layer_normalization: برای نرمال‌سازی لایه‌ها در مدل ترنسفورمر استفاده می‌شه تا پایداری آموزش رو بهتر کنه.

#### multi_head_attention: مکانیزمی برای توجه چندسر (multi-head attention) که در ترنسفورمر برای تمرکز روی قسمت‌های مختلف ورودی استفاده می‌شه.

#### multi_head_cross_attention: نوع خاصی از توجه که بین رمزگذار و رمزگشا کار می‌کنه.

#### positional_encoding: اضافه کردن اطلاعات موقعیت به بردارها، چون ترنسفورمر ترتیب ورودی‌ها رو به صورت ذاتی نمی‌فهمه.

#### self_attention: نوع خاصی از توجه که هر بخش ورودی با خودش مقایسه می‌شه.

#### train:  آموزش مدل

#### transformer: فایل اصلی که  معماری ترنسفورمر رو پیاده‌سازی می‌کنه.

# import libraries :

In [1]:
from transformer import Transformer
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import importlib

# preprocessing :

In [2]:
START_TOKEN = '<s>'
END_TOKEN = '<\s>'
PADDING_TOKEN = '<pad>'
english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                    ':', '<', '=', '>', '?', '@', ';',
                    '[', '\\', ']',
                    '^', '_', '`',
                    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                    'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                    'y', 'z',
                    '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN
                    ]

persian_vocabulary = [
START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
'0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';',
':', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`',
'آ', 'ا', 'ب', 'پ', 'ت', 'ث', 'ج', 'چ', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'ژ', 'س', 'ش',
'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ک', 'گ', 'ل', 'م', 'ن', 'و', 'ه', 'ی',
'ء', 'ۀ', 'ؤ', 'ي', 'ك', 'ة', '‌', 'ٔ', 'ى', PADDING_TOKEN, END_TOKEN
]
index_to_persian = {k:v for k,v in enumerate(persian_vocabulary)}
persian_to_index = {v:k for k,v in enumerate(persian_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [3]:
df = pd.read_excel(r'C:\\Users\\ExoGame\\OneDrive\\Desktop\\New folder (8)\\English2Persian-translation-main\\dataset\\shortened_dataset.xlsx')
df.head()

,Unnamed: 0,persian,english
0,0,گلدان روی میز چای حاضر و آماده بود.,the vase filled with water was ready in the ce...
1,1,آن وقت قاضی چه کرد؟,What did the justice do?
2,2,به روزگار فيلماي ؛ نقطه تلاقي ؛ ماري کثيف يا ه...,"vanishing point days , the dirty mary crazy la..."
3,3,افراد مورد اعتماد زیردستهایشان به عنوان سرپرست...,with the trust of his subordinates as the head...
4,4,زودتر برویم. من حاضرم.,"I am ready, my son, said Mercedes."


In [4]:
df['english'] = df['english'].astype(str)
df['persian'] = df['persian'].astype(str)

In [5]:
def helper_english(x:str):
        for c in x:
            if not c in english_vocabulary:
                x = x.replace(c, '')
        return x

def helper_persian(x:str):
    for c in x:
        if not c in persian_vocabulary:
            x = x.replace(c, '')
    return x

df['english'] = df['english'].apply(str.lower)
df['english'] = df['english'].apply(helper_english)
df['persian'] = df['persian'].apply(helper_persian)
persian_sentences = df['persian'].to_list()
english_sentences = df['english'].to_list()
enlish_sentences = df['english'].to_list()
persian_sentences = df['persian'].to_list()

# 5-Layer Transformer Implementation for Persian :

In [6]:
model_dim = 512
#خروجی لایه‌های مدل. یعنی هر توکن به یک بردار 512 بعدی نگاشت می‌شود
batch_size = 64
#تعداد نمونه‌هایی که همزمان پردازش می‌شوند
hidden_fc = 2048
#مشخص‌کننده اندازه لایه پنهان در پرسپترون چندلایه
num_heads = 8
#تعداد هدهای (heads) در مکانیزم توجه چندسر (Multi-Head Attention)
drop_prob = 0.1
#نرخ regularization
num_layers = 5
#تعداد لایه‌های ترنسفورمر (هم انکودر و هم دیکودر).
max_sequence_length = 200
#حداکثر طول دنباله‌ای که مدل می‌تواند پردازش کند
persian_vocab_size = len(persian_vocabulary)

transformer = Transformer((batch_size, max_sequence_length, model_dim),
                          model_dim,
                          hidden_fc,
                          num_heads,
                          drop_prob,
                          num_layers,
                          max_sequence_length,
                          persian_vocab_size,
                          english_to_index,
                          persian_to_index,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)

In [7]:
class TranslateDataset(Dataset):
    def __init__(self, english_sentences, persian_sentences):
        super().__init__()
        self.english_sentences = english_sentences
        self.persian_sentences = persian_sentences

    def __len__(self):
        return len(self.persian_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.persian_sentences[idx]


dataset = TranslateDataset(english_sentences, persian_sentences)
train_loader = DataLoader(dataset, batch_size)

In [8]:
criterion = nn.CrossEntropyLoss(ignore_index=persian_to_index[PADDING_TOKEN])
#Loss Function
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)
# اگر پارامتر بیش از یک بعد داشته باشد، این خط از روش مقداردهی اولیه Xavier Uniform برای مقداردهی وزن‌ها استفاده می‌کند
optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
# ایجاد بهینه‌ساز Adam
#  برای ‌روزرسانی وزن‌ها (learning rate) را روی 0.0001
device = torch.device('cuda')
# محاسباتی را روی GPU (با استفاده از CUDA)

# Attention Masks


 سه نوع ماسک مختلف برای مدل ترنسفورمر ایجاد می‌کند تا پردازش ترجمه را به درستی انجام شود

#### ماسک انکودر (encoder_self_attention_mask):



##### پدینگ‌های جمله انگلیسی را پنهان می‌کند
###### به مدل می‌گوید کدام قسمت‌های ورودی را نادیده بگیرد 







### ماسک دیکودر (decoder_self_attention_mask):


###### ترکیبی از دو چیز است
###### پدینگ‌های جمله فارسی
###### ماسک "نگاه به جلو" (look-ahead) که از دیدن کلمات آینده جلوگیری می‌کند







### ماسک توجه متقاطع (decoder_cross_attention_mask):



###### ارتباط بین پدینگ‌های فارسی و کلمات انگلیسی را مسدود می‌کند

### مثال
جمله انگلیسی: ["I", "love", پدینگ, پدینگ, پدینگ] 

جمله فارسی: ["من", "عاشق", "هستم", پدینگ, پدینگ]

ماسک انکودر به مدل می‌گوید: "به سه کلمه اول توجه کن، دو تای آخر را نادیده بگیر"
ماسک دیکودر می‌گوید: "فقط به کلمات قبلی نگاه کن و پدینگ‌ها را نادیده بگیر"
 ماسک توجه متقاطع میگه وقتی دارم کلمه فارسی را تولید می‌کنم، به کدام کلمات انگلیسی توجه کنم یا نکنم


In [3]:

NEG_INFTY = -1e9
#مقدار پرکننده برای بخش‌هایی از ماسک استفاده می‌شود که مدل نباید به آن‌ها توجه کند



def create_masks(eng_batch, persian_batch, number_of_heads):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length], True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)

    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length], False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length], False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length], False)

    for idx in range(num_sentences):
        eng_sentence_length, persian_sentence_length = len(eng_batch[idx]), len(persian_batch[idx])
        eng_chars_to_padding_mask = np.arange(eng_sentence_length, max_sequence_length)
        persian_chars_to_padding_mask = np.arange(persian_sentence_length, max_sequence_length)

        encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
        encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True

        decoder_padding_mask_self_attention[idx, persian_chars_to_padding_mask, :] = True
        decoder_padding_mask_self_attention[idx, :, persian_chars_to_padding_mask] = True

        decoder_padding_mask_cross_attention[idx, persian_chars_to_padding_mask, :] = True
        decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True


    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask = torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)


    encoder_self_attention_mask = encoder_self_attention_mask.unsqueeze(1).repeat(1, number_of_heads, 1, 1)
    decoder_self_attention_mask = decoder_self_attention_mask.unsqueeze(1).repeat(1, number_of_heads, 1, 1)
    decoder_cross_attention_mask = decoder_cross_attention_mask.unsqueeze(1).repeat(1, number_of_heads, 1, 1)

    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask


# training :

In [10]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 20
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, per_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, per_batch, num_heads)
        optim.zero_grad()
        persian_predictions = transformer(eng_batch,
                                     per_batch,
                                     encoder_self_attention_mask.to(device),
                                     decoder_self_attention_mask.to(device),
                                     decoder_cross_attention_mask.to(device),
                                     encoder_start_token=False,
                                     encoder_end_token=False,
                                     decoder_start_token=True,
                                     decoder_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(per_batch, start_token=False, end_token=True)
        loss = criterion(
            persian_predictions.view(-1, persian_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == persian_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        if batch_num % 100 == 0:
            print(f"loss at batch number {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Persian Translation: {per_batch[0]}")
            persian_sentence_predicted = torch.argmax(persian_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in persian_sentence_predicted:
              if idx == persian_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_persian[idx.item()]
            print(f"Persian Prediction: {predicted_sentence}")
            print("-------------------------------------------")




Epoch 1
loss at batch number 0 : 0.0013898820616304874
English: the vase filled with water was ready in the center of the tea table.
Persian Translation: گلدان روی میز چای حاضر و آماده بود.
Persian Prediction: ءخ
-------------------------------------------
loss at batch number 100 : 0.0011919466778635979
English: but even here, while on the scaffolding, in the midst of a talk explaining the future arrangements of the house, he interrupted himself:
Persian Translation: ولی در میان این سخنان نیز هنگامی که نقشه خانه را برای پی‌یر توضیح می‌داد یک مرتبه روی چوب بست ایستاده گفت:
Persian Prediction: اا                                                                      د                     ا   مریها تد  ا  ر د  ک د6ت د
-------------------------------------------
loss at batch number 200 : 0.0007677569519728422
English: don't play with me like a cat with a mouse.
Persian Translation: چرا مثل گربه‌ای که با موش بازی کند با من رفتار می‌کنید
Persian Prediction: اا                                